# Table of Content
* [Goal](#Goal)
* [Wrangle](#Wrangle)
* [Explore](#Explore)
* [Model](#Model)

In [2]:
import numpy as np
from bs4 import BeautifulSoup
import requests
from requests import get
import pandas as pd
import numpy as np
import os
import time
import re
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from prepare import prep_gitMDs

print('Initalized')

Initalized


# Goal
The goal of this project is to use Natural Language Processing (NLP) to predict the primary coding language of a gitrepo using the repo's README.  

# Wrangle
We will use the modules ***[acquire.py](https://github.com/NLP-MVPs/nlp-project/blob/main/acquire.py)*** and ***[prepare.py](https://github.com/NLP-MVPs/nlp-project/blob/main/prepare.py)*** to gather the data and put it into a dataframe. From there we will begin to wrangle the data for exploration focusining mainly on creating word strings from our data.

[Table of Content](#Table-of-Content)

In [ ]:
# Use the 

# Explore
[Table of Content](#Table-of-Content)

# Model
[Table of Content](#Table-of-Content)